In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from articulate_anything.utils.metric import compute_link_placement_result, analyze_link_placement_by_type, compute_joint_pred_result, analyze_joint_pred_by_type

pybullet build time: Nov 28 2023 23:52:03


In [4]:
result_dir = "results"
dataset_dir = "datasets/partnet-mobility-v0/dataset"
strategy="gt"

In [5]:
link_diffs = compute_link_placement_result(result_dir, strategy=strategy,)
type_results = analyze_link_placement_by_type(link_diffs)

FileNotFoundError: [Errno 2] No such file or directory: 'results/10/meta.json'

In [ ]:
joint_diffs = compute_joint_pred_result(result_dir, strategy=strategy)
joint_results = analyze_joint_pred_by_type(joint_diffs)

In [ ]:
import random
from articulate_anything.utils.partnet_utils import track_obj_types, sample_obj_id
from collections import Counter

def filter_objects_by_link_diff(link_diffs, threshold=0.05):
    return [obj_id for obj_id, data in link_diffs.items() if data['link_diff']['average_diff'] < threshold]

def sample_proportional_to_category_size_filtered(real2code_categories, num_samples, filtered_obj_ids):
    obj_types = track_obj_types()
    
    # Filter obj_types to only include objects in filtered_obj_ids
    filtered_obj_types = {cat: [obj for obj in objs if obj in filtered_obj_ids] 
                          for cat, objs in obj_types.items()}
    
    total_objects = sum(len(filtered_obj_types[cat]) for cat in real2code_categories if cat in filtered_obj_types)
    
    if total_objects == 0:
        raise ValueError("No objects meet the filtering criteria")
    
    sampled_objects = []
    for category in real2code_categories:
        if category in filtered_obj_types:
            proportion = len(filtered_obj_types[category]) / total_objects
            num_category_samples = int(num_samples * proportion)
            category_samples = random.sample(filtered_obj_types[category], 
                                             min(num_category_samples, len(filtered_obj_types[category])))
            sampled_objects.extend(category_samples)
    
    # Add any remaining samples due to rounding
    while len(sampled_objects) < num_samples:
        category = random.choice([cat for cat in real2code_categories if cat in filtered_obj_types])
        remaining_objects = [obj for obj in filtered_obj_types[category] if obj not in sampled_objects]
        if remaining_objects:
            extra_sample = random.choice(remaining_objects)
            sampled_objects.append(extra_sample)
        else:
            break  # Break if we've exhausted all available objects
    
    return sampled_objects

def train_val_split_filtered(real2code_categories, num_trains, num_vals, link_diffs):
    total_samples = num_trains + num_vals
    
    # Filter objects based on link difference
    filtered_obj_ids = filter_objects_by_link_diff(link_diffs)
    
    # Sample all objects at once
    all_samples = sample_proportional_to_category_size_filtered(real2code_categories, total_samples, filtered_obj_ids)
    
    # Shuffle the samples
    random.shuffle(all_samples)
    
    # Split into train and val sets
    train_samples = all_samples[:num_trains]
    val_samples = all_samples[num_trains:]
    
    return train_samples, val_samples

def print_category_distribution(obj_ids, real2code_categories, set_name):
    category_counts = Counter()
    obj_types = track_obj_types()
    for obj_id in obj_ids:
        for category in real2code_categories:
            if obj_id in obj_types.get(category, []):
                category_counts[category] += 1
                break
        else:
            category_counts['Other'] += 1
    
    total_objects = len(obj_ids)
    print(f"Distribution of objects per category ({set_name} set):")
    print("-------------------------------------")
    for category in real2code_categories:
        count = category_counts[category]
        percentage = (count / total_objects) * 100 if total_objects > 0 else 0
        print(f"{category}: {count} ({percentage:.2f}%)")
    if category_counts['Other'] > 0:
        other_count = category_counts['Other']
        other_percentage = (other_count / total_objects) * 100 if total_objects > 0 else 0
        print(f"Other: {other_count} ({other_percentage:.2f}%)")
    print(f"Total: {total_objects}")

# Usage
real2code_categories = ["StorageFurniture", "Laptop", "Box", "Table", "Refrigerator"]
num_trains = 467
num_vals = 35

# Assuming link_diffs is already computed
train_obj_ids, val_obj_ids = train_val_split_filtered(real2code_categories, num_trains, num_vals, link_diffs)

print_category_distribution(train_obj_ids, real2code_categories, "Training")
print("\n")
print_category_distribution(val_obj_ids, real2code_categories, "Validation")

In [ ]:
def generate_tasks_from_joint_diffs(joint_diffs, sample_set):
    tasks = []
    for key, value in joint_diffs.items():
        obj_id, joint_id, semantic_joint_id = key
        if obj_id in sample_set and value["failure_reason"] == "success":
            task = {
                "obj_id": obj_id,
                "joint_id": joint_id,
                "semantic_joint_id": semantic_joint_id,
                "joint_diff": value,
                "link_diff": link_diffs[obj_id],
            }
            tasks.append(task)
    return tasks

In [ ]:
train_tasks = generate_tasks_from_joint_diffs(joint_diffs, train_obj_ids)
val_tasks = generate_tasks_from_joint_diffs(joint_diffs, val_obj_ids)
len(train_tasks), len(val_tasks)

In [ ]:
from articulate_anything.utils.utils import save_json
save_json(train_tasks, "train_tasks.json")
save_json(val_tasks, "val_tasks.json")